#### **INSTALL THƯ VIỆN LIÊN QUAN**

In [ ]:
! pip install torchtext==0.17.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
! pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00


#### **KẾT NỐI GOOGLE DRIVE VÀ IMPORT THƯ VIỆN**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pprint
import numpy as np
import pandas as pd

from sklearn import preprocessing, metrics

import torch
from torch import Tensor
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torchtext
import torchtext.transforms as T
import torchtext.functional as F
from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER

import pickle

from underthesea import sent_tokenize, word_tokenize
from underthesea import ner
import nltk
import re
import string
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

#### **THIẾT LẬP CÁC HÀM SENTIMENT ANALYSIS**

In [ ]:
'''with open("stopwords-vi.txt", "r", encoding="utf-8") as file:
    stopwords = file.read().split("\n")
stopwords = set(stopwords)'''

def Name_Entity_Recognize(text):
    doc = ner(text)
    temp = [i[0] for i in doc if i[1] in ['CH', 'Np', 'M', 'Nu', 'X', 'R']]
    ent = []
    for i in temp:
        i = str(i)
        i = i.lower()
        ent.append(i)
    return ent

def clean_text(text):
    words = set(nltk.corpus.words.words())
    text = re.sub(r'\([A-Z\-0-9]{2,}\)?', '', text)
    #ent = Name_Entity_Recognize(text)
    text = word_tokenize(text)
    #text = [w.lower() for w in text if w.lower() not in stopwords]
    #text = [w for w in text if w not in ent]
    text = [w for w in text if len(w)!=1]
    text = [w for w in text if w not in words]
    text = [re.sub(r'[^\w\s]','', w) for w in text]## Loại bỏ dấu câu
    text = [re.sub(r'\b\d+\b', '', w) for w in text] ## Xóa số
    text = [re.sub(r'[\d]', '', w) for w in text] ## Xóa số
    text = [re.sub(r'\s\s+', '', w) for w in text] ## Cắt bỏ khoảng trắng
    text = [w for w in text if w != '']
    text = [w.replace(' ', '_') for w in text]
    text = ' '.join(c for c in text if c <= '\uFFFF')
    return text

In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
def prepare_model():
    num_classes = 3
    input_dim = 768

    classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
    model = XLMR_BASE_ENCODER.get_model(head=classifier_head)

    DEMO_MODEL_PATH = '/content/drive/MyDrive/trained_xlmRoBertamodel/model_max_accuracy.pth'
    model.load_state_dict(torch.load(DEMO_MODEL_PATH, map_location=torch.device('cpu')))
    #model.to(DEVICE)

    print(f'Loaded model in [{DEMO_MODEL_PATH}]')
    return model

In [ ]:
def prepare_text_transform():
    text_transform = torchtext.models.XLMR_LARGE_ENCODER.transform()
    return text_transform

In [ ]:
def predict(sentence, model, text_transform):
    transformed_text = text_transform(sentence)
    out = model(torch.tensor([transformed_text]).to(DEVICE))
    return out, label_map[torch.argmax(out).item()]

In [ ]:
label_map = {
    0: 'negative',
    1: 'neutral',
    2: 'postive'
}

In [ ]:
model = prepare_model()
text_transform = prepare_text_transform()

Downloading: "https://download.pytorch.org/models/text/xlmr.base.encoder.pt" to /root/.cache/torch/hub/checkpoints/xlmr.base.encoder.pt
100%|██████████| 1.03G/1.03G [00:05<00:00, 198MB/s]


Loaded model in [/content/drive/MyDrive/trained_xlmRoBertamodel/model_max_accuracy.pth]


100%|██████████| 5.07M/5.07M [00:00<00:00, 12.6MB/s]
Downloading: "https://download.pytorch.org/models/text/xlmr.vocab.pt" to /root/.cache/torch/hub/checkpoints/xlmr.vocab.pt
100%|██████████| 4.85M/4.85M [00:00<00:00, 9.52MB/s]


#### **SENTIMENT CÁC TITLE**

In [ ]:
df = pd.DataFrame()
for i in [18831, 18832, 18833, 18834, 18835, 18836, 18839]:
  df_temp = pd.read_csv(f'/content/drive/MyDrive/Crawl Data - Code/{i}_all_data.csv')
  df_temp = df_temp.drop(columns=['Unnamed: 0'])
  df = pd.concat([df, df_temp])
df = df.reset_index(drop=True)
df

,Title,Link,Time
0,Chứng khoán VIX: Kết quả quý III tăng trưởng h...,https://cafef.vn/chung-khoan-vix-ket-qua-quy-i...,2024-10-08
1,Em họ hối hận khi giúp Trương Mỹ Lan chiếm đoạ...,https://cafef.vn/em-ho-hoi-han-khi-giup-truong...,2024-10-08
2,Công ty chứng khoán của Chủ tịch 9x thế hệ F2 ...,https://cafef.vn/cong-ty-chung-khoan-cua-chu-t...,2024-10-08
3,Xe điện đối thủ của VinFast VF 3 sẽ bán với gi...,https://cafef.vn/xe-dien-doi-thu-cua-vinfast-v...,2024-10-08
4,"Tập đoàn Than - Khoáng sản thu 123,5 nghìn tỷ ...",https://cafef.vn/tap-doan-than-khoang-san-thu-...,2024-10-08
...,...,...,...
534520,80.000 tấn dầu thô đầu tiên đã được bơm vào nh...,https://cafef.vn/hang-hoa-nguyen-lieu/80000-ta...,2008-12-06
534521,Dệt may Pháp kêu gọi tăng cường kiểm tra hàng ...,https://cafef.vn/hang-hoa-nguyen-lieu/det-may-...,2008-12-05
534522,"Dự kiến xuất khẩu cao su 2008 đạt 1,8 tỷ USD",https://cafef.vn/hang-hoa-nguyen-lieu/du-kien-...,2008-12-05
534523,10 tháng nhập khẩu phân bón giảm 6% so cùng kỳ,https://cafef.vn/hang-hoa-nguyen-lieu/10-thang...,2008-12-05


In [ ]:
df['Time'] = pd.to_datetime(df['Time'])
df_result = df.groupby(df['Time'].dt.date)['Title'].apply(lambda x: '. '.join(x)).reset_index()
df_result = df_result[(df_result.Time < pd.to_datetime('2015-01-01').date())]
df_result = df_result.reset_index(drop=True)
df_result

,Time,Title
0,2007-01-11,Tiếp tục giảm thuế nhiều mặt hàng
1,2007-10-03,SSI góp phần làm VN-Index mất điểm. TSC phát h...
2,2007-10-16,Cổ phiếu ngân hàng: Lên sàn có cứu được giá?
3,2007-10-18,VFM chỉ là thất tín?. Đấu thầu 700 tỷ đồng trá...
4,2007-10-19,Luật chưa có nghị định: Ai chờ ai?. Tập đoàn D...
...,...,...
2626,2014-12-27,Em gái ông Nguyễn Quốc Cường vào Top U40 giàu ...
2627,2014-12-28,"Chứng khoán tuần qua: Qua 1 tuần uể oải, hiệu ..."
2628,2014-12-29,[Chứng khoán 29/12] Phiên ATC chiếm 1/3 giá tr...
2629,2014-12-30,Chứng khoán HSC bị phạt do bố trí người chưa c...


In [ ]:
result = []
titles = df_result.Title.to_list()
times = df_result.Time.to_list()
for i in range(len(titles)):
  print(i)
  prediction = predict(titles[i], model, text_transform)
  neg = prediction[0][0,0].item()
  neu = prediction[0][0,1].item()
  pos = prediction[0][0,2].item()
  label = prediction[1]
  result.append([times[i], neg, neu, pos])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
df = pd.DataFrame(result, columns=['Time', 'Negative', 'Neutral', 'Positive'])
df.to_csv('Data/Sentiment Data/all_labeled.csv')